In [1]:
import polars as pl
import altair as alt

In [57]:
slovenske = [
    "S SK 1939",
    "S SR 1939-1945"
]

In [85]:
df = pl.read_parquet("data_raw/smazat_projevy.parquet").sort(by='datum').with_columns(
    pl.col('text').str.len_bytes().alias('delka')
).filter(
    ~pl.col('komora_komplet').str.contains('SNR')
).filter(
    ~pl.col('komora_komplet').str.contains('SNR')
).filter(
    ~pl.col('komora_komplet').str.contains('NR SR')
).filter(
    ~pl.col('komora_komplet').is_in(slovenske)
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.first().str.split(')').list.first().str.to_lowercase().str.strip_chars().alias('poznamka_mluvciho')
)

In [39]:
pl.Config.set_tbl_rows(50)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [79]:
df.filter(pl.col('poznamka_mluvciho').str.contains('sky')).group_by('poznamka_mluvciho').len().sort(by='len',descending=True).head(30)

poznamka_mluvciho,len
str,u32
"""maďarsky""",399
"""rusky""",93
"""malorusky""",64
"""polsky""",47
"""slovensky""",8
"""velkorusky""",3
"""posl. kurťak [malorusky]""",2
"""pokračuje slovensky""",2
"""posl. dr markovič [maďarsky]""",1


In [87]:
jazyky = [
    'německy',
    'maďarsky',
    'rusky',
    'malorusky',
    'polsky',
]

In [111]:
vyvoj_jazyku = []
for j in jazyky:
    vyvoj_jazyku.append(df.filter(pl.col('poznamka_mluvciho') == j).group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).with_columns(pl.lit(j).alias('jazyk')))

vyvoj_jazyku = pl.concat(vyvoj_jazyku)

In [117]:
alt.Chart(vyvoj_jazyku.to_pandas(), width=500, height=100).mark_bar().encode(alt.X('datum:T'),alt.Y('text:Q'),alt.Row('jazyk:N'))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [129]:
df.filter(pl.col('mluvci').str.contains('[Bb]ulhar'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,delka,poznamka_mluvciho
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,u32,str
"""Vedoucí bulharské delegace Bojan Balgaranov (uvítán povstáním a hlučným potleskem)""","""1954ns_stenprot_018schuz_s018001.htm""",null,"""Soudruhu předsedo Národního shromáždění republiky Československé! Soudružky a soudruzi poslanci! Jsem šťasten, že mohu poděkovat Národnímu shromáždění…",3,"""ns""",1954,18,1,1957-04-18,"""NS RČS 1954-1960""","""18. schůze""",true,true,12389,"""uvítán povstáním a hlučným potleskem"""
"""Velvyslanec Bulharské republiky Martin Tomov""","""2004se_stenprot_007schuz_s007038.htm""",null,"""Vážený pane předsedo, dámy a pánové, vážený Senáte. Jménem Bulharské republiky vám děkuji za příznivé hlasování, jednomyslné hlasování. Já jsem přesvě…",11,"""se""",2004,7,38,2005-09-15,"""PČR, Senát 2004-2006""","""7. schůze""",true,true,898,null


In [109]:
pl.concat(vyvoj_jazyku)

datum,text,jazyk
date,u32,str
1920-01-01,207,"""německy"""
1921-01-01,391,"""německy"""
1922-01-01,323,"""německy"""
1923-01-01,210,"""německy"""
1924-01-01,185,"""německy"""
1925-01-01,108,"""německy"""
1926-01-01,236,"""německy"""
1927-01-01,200,"""německy"""
1928-01-01,198,"""německy"""


In [83]:
df.group_by('poznamka_mluvciho').len().sort(by='len',descending=True).head(50)

poznamka_mluvciho,len
str,u32
null,671403
"""zvoní""",16917
"""čte""",12340
"""pokračuje""",5585
"""německy""",3378
"""maďarsky""",399
"""podávaje předsedovi ruku""",286
"""podávaje ministerskému předsedovi ruku""",136
"""rep.""",116


In [27]:
df.filter(pl.col("mluvci").str.contains(r"německy"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Posl. dr. Lodgman (německy)""","""1920ns_ps_stenprot_002schuz_s0…",null,"""Dámy a pánové! (Hluk.)""",3,"""ns""",1920,2,2,1920-06-01,"""Národní shromáždění českoslove…","""2. schůze""",true,true
"""Poslanec Seliger (německy)""","""1920ns_ps_stenprot_003schuz_s0…",null,"""Dámy a pánové! Dříve nežli bud…",2,"""ns""",1920,3,3,1920-06-02,"""Národní shromáždění českoslove…","""3. schůze""",true,true
"""Posl. Křepek (německy)""","""1920ns_ps_stenprot_003schuz_s0…",null,"""Velectěné dámy a pánové! Ve vč…",8,"""ns""",1920,3,1,1920-06-02,"""Národní shromáždění českoslove…","""3. schůze""",true,true
"""Poslanec Wittich (německy)""","""1920ns_ps_stenprot_004schuz_s0…",null,"""Slavná sněmovno! Jménem svých …",2,"""ns""",1920,4,8,1920-06-08,"""Národní shromáždění českoslove…","""4. schůze""",true,true
"""Senátor Jelinek (německy)""","""1920ns_se_stenprot_003schuz_s0…",null,"""Dámy a pánové! Předem rád bych…",2,"""ns""",1920,3,7,1920-06-09,"""NS RČS 1920-1925""","""3. schůze""",true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sen. H. Müller (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Jménem klubu ně…",3,"""ns""",1935,119,2,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true
"""Sen. Reil (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Předložená nove…",4,"""ns""",1935,119,2,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true
"""Sen. Liehm (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Aby bylo možno …",5,"""ns""",1935,119,1,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true


In [15]:
df.filter(pl.col("text").str.contains(r"\([Rr]usínsk"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
null,"""1935ns_ps_stenprot_142schuz_s1…",null,"""Není-li školní docházka doposu…",1,"""ns""",1935,142,2,1938-03-18,"""NS RČS 1935-1938""","""142. schůze""",true,true
"""Posl. Egri""","""1964ns_stenprot_028schuz_s0280…",null,"""Vážený súdruh prezident, súdru…",2,"""ns""",1964,28,9,1968-10-27,"""NS RČS 1964-1968""","""28. schůze""",true,true
"""Ministr spravedlnosti poslanec…","""1964ns_stenprot_028schuz_s0280…",null,"""Vážený soudruhu presidente, vá…",2,"""ns""",1964,28,8,1968-10-27,"""NS RČS 1964-1968""","""28. schůze""",true,true
"""Poslankyně SN E. Vitézová""","""1986fs_slsn_stenprot_028schuz_…",null,"""Za politické hnutie Egyűttelés…",4,"""fs""",1986,28,15,1990-05-03,"""Federální shromáždění ČSSR, 19…","""28. schůze""",true,true


In [33]:
df.filter(pl.col("mluvci").str.contains(r"německy"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Posl. dr. Lodgman (německy)""","""1920ns_ps_stenprot_002schuz_s0…",null,"""Dámy a pánové! (Hluk.)""",3,"""ns""",1920,2,2,1920-06-01,"""Národní shromáždění českoslove…","""2. schůze""",true,true
"""Poslanec Seliger (německy)""","""1920ns_ps_stenprot_003schuz_s0…",null,"""Dámy a pánové! Dříve nežli bud…",2,"""ns""",1920,3,3,1920-06-02,"""Národní shromáždění českoslove…","""3. schůze""",true,true
"""Posl. Křepek (německy)""","""1920ns_ps_stenprot_003schuz_s0…",null,"""Velectěné dámy a pánové! Ve vč…",8,"""ns""",1920,3,1,1920-06-02,"""Národní shromáždění českoslove…","""3. schůze""",true,true
"""Poslanec Wittich (německy)""","""1920ns_ps_stenprot_004schuz_s0…",null,"""Slavná sněmovno! Jménem svých …",2,"""ns""",1920,4,8,1920-06-08,"""Národní shromáždění českoslove…","""4. schůze""",true,true
"""Senátor Jelinek (německy)""","""1920ns_se_stenprot_003schuz_s0…",null,"""Dámy a pánové! Předem rád bych…",2,"""ns""",1920,3,7,1920-06-09,"""NS RČS 1920-1925""","""3. schůze""",true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sen. H. Müller (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Jménem klubu ně…",3,"""ns""",1935,119,2,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true
"""Sen. Reil (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Předložená nove…",4,"""ns""",1935,119,2,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true
"""Sen. Liehm (německy)""","""1935ns_se_stenprot_119schuz_s1…",null,"""Slavný senáte! Aby bylo možno …",5,"""ns""",1935,119,1,1938-05-18,"""NS RČS 1935-1938""","""119. schůze""",true,true


In [35]:
df.filter(pl.col("mluvci").str.contains(r"maďarsky"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Posl. Szentiványi (podávaje př…","""1920ns_ps_stenprot_002schuz_s0…",null,"""Slibuji. (Heil! Éljen!)""",13,"""ns""",1920,2,2,1920-06-01,"""Národní shromáždění českoslove…","""2. schůze""",true,true
"""Posl. dr. Ékes Körmendy (maďar…","""1920ns_ps_stenprot_003schuz_s0…",null,"""Ctěné dámy a pánové! Mám cest …",11,"""ns""",1920,3,1,1920-06-02,"""Národní shromáždění českoslove…","""3. schůze""",true,true
"""Posl. Surányi (maďarsky)""","""1920ns_ps_stenprot_004schuz_s0…",null,"""Slavné shromáždění! Myslím, že…",10,"""ns""",1920,4,1,1920-06-08,"""Národní shromáždění českoslove…","""4. schůze""",true,true
"""Senátor Dr. Koperniczky (maďar…","""1920ns_se_stenprot_004schuz_s0…",null,"""Ctěný senáte! Skutečnost, že o…",5,"""ns""",1920,4,4,1920-06-10,"""NS RČS 1920-1925""","""4. schůze""",true,true
"""Posl. dr. Lelley (maďarsky)""","""1920ns_ps_stenprot_009schuz_s0…",null,"""Pánové a dámy! Když v této sně…",2,"""ns""",1920,9,4,1920-06-18,"""Národní shromáždění českoslove…","""9. schůze""",true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Sen. dr Turchányi (maďarsky)""","""1935ns_se_stenprot_111schuz_s1…",null,"""Veľavážený senát! Vláda požadu…",11,"""ns""",1935,111,1,1938-04-06,"""NS RČS 1935-1938""","""111. schůze""",true,true
"""Sen. Hokky (maďarsky)""","""1935ns_se_stenprot_112schuz_s1…",null,"""Vážený senát! K osnovám, ktorý…",12,"""ns""",1935,112,4,1938-04-07,"""NS RČS 1935-1938""","""112. schůze""",true,true
"""Sen. dr Pajor (maďarsky)""","""1935ns_se_stenprot_113schuz_s1…",null,"""Vážený senát! Už za pojednávan…",3,"""ns""",1935,113,2,1938-04-08,"""NS RČS 1935-1938""","""113. schůze""",true,true


In [67]:
df

jazyk,len
str,u32
null,671403
"""zvoní""",16917
"""čte""",12340
"""pokračuje""",5585
"""německy""",3378
"""maďarsky""",399
"""podávaje předsedovi ruku""",286
"""podávaje ministerskému předsedovi ruku""",136
"""rep.""",116


In [41]:
df.filter(pl.col("mluvci").str.contains(r"německy")).select(pl.col("mluvci")).head(1).item()

'Posl. dr. Lodgman (německy)'

In [17]:
df.filter(pl.col("text").str.contains(r"\([Rr]usínsk")).select(pl.col("text")).head(1).item()

'Není-li školní docházka doposud upravena naším zákonem, není to o nic lepší s organisací školních řadů na Podkarpatské Rusi. Byly stvořeny ad hoc, nemají ještě dodnes žádného zákonitého podkladu a jejich úprava nebyla provedena ani doposud. Dokonce se tvrdí, že ani užhorodský referát ministerstva školství nemá zřizovací listiny, o školních inspektorátech ani nemluvě. Je proto v zájmu veřejného pořádku a veřejné správy na Podkarpatské Rusi vůbec, aby se s touto věcí hnulo. Svého času jsme se těšili, že budou projednány Dérerovy osnovy zákonů o školské správě, avšak ani tyto osnovy, ani osnovy novější se nemohou dostati kupředu, snad pro odpor některé strany, a tak nám nezbude nic jiného, než aby tyto věci byly pro Podkarpatskou Rus upraveny zvlášť, do té doby, než bude zřízen podkarpatský sněm. Jsem si vědom, že tento můj názor narazí na odpor se strany oposice, ale mám za to, že když jde o zájem vyšší, když běží o rozvoj celého školství v zemi a jeho další vývoj, musíme od některých d